## _Medical Insurance Costs_

Pada kasus ini, terdapat data tentang informasi kesehatan dan biaya yang harus dikeluarkan oleh asuransi kesehatan. Infomasi terkait dengan data _medical insurance cost_ adalah sebagai berikut,

1. Age: Usia penerima manfaat
2. Sex: Gender penerima manfaat (_male_, _femele_)
3. Bmi : Body Mass Index
4. Children: Jumlah anak/tanggungan yang dicover oleh pihak asuransi
5. Smoker: Status perokok (_yes_, _no_)
6. Region: Wilayah tempat tinggal penerima manfaat
7. Charges: Biaya yang dikeluarkan oleh asuransi

In [1]:
# Selayang pandang data Medical Insurance Costs
import pandas as pd

df = pd.read_csv('data/insurance.csv')

display(df.head())

display(df.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


### Tantangan

Buatlah model regresi untuk memprediksi biaya yang harus dikeluarkan oleh pihak asuransi berdasarkan data. Validasi performa model regresi Anda dengan nilai ***R-squared ($R^2$)***

#### _Tasks_

1. Pastikan semua variabel kategorial diolah dengan baik. (Gunakan fitur mapping pada pandas)
2. Cek kondisi multicollinearity untuk semua variabel independen. Jika ada, antar variabel apakah itu?
3. Pastikan model menggunakan variabel yang tidak memiliki nilai multicollinearity yang tinggi
4. (Hints) Anda dapat menggunakan nilai ***Variance Inflation Factor (VIF)*** untuk mengetahui tingkat multicollinearity pada sebuah variabel independent.
5. Evaluasi model yang Anda buat dengan nilai $R^2$
6. Simpulkan, variabel independen apa saja yang dapat digunakan untuk menghasilkan model regresi yang baik pada kasus _medical insurance costs_?

In [6]:
from sklearn.preprocessing import LabelEncoder

# Inisiasi label encoder
encode = LabelEncoder()

# Terpakan label encoder
df['sex'] = encode.fit_transform(df['sex'])
df['smoker'] = encode.fit_transform(df['smoker'])
df['region'] = encode.fit_transform(df['region'])

# Cek hasil
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# the independent variables set
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

    feature        VIF
0       age   7.551348
1       sex   2.001061
2       bmi  10.371829
3  children   1.801245
4    smoker   1.256837
5    region   2.924528


Dari hasil kalkulasi VIF di atas, dapat dilihat bahwa BMI dan age memiliki hasil yang cukup tinggi yang menyatakan bahwa dua data tersebut cukup berkaitan erat. Hal ini sangat benar, karena angka BMI seseorang dapat dipengaruhi oleh usia individu tersebut. Pada umumnya, semakin tua umur seseorang maka akan semakin besar juga kadar lemak pada tubuhnya yang menyebabkan meningkatnya angka BMI individu tersebut. 

Maka dari itu kita akan menyingkirkan variabel BMI dan age dari model, karena itu dapat menghasilkan hasil yang kurang akurat untuk model regresi yang akan kita buat saat ini.

In [34]:
X=df.iloc[:,:6]
y=df.iloc[:,6].values
X.drop(labels=['age', 'bmi'], axis=1)

,sex,children,smoker,region
0,0,0,1,3
1,1,1,0,2
2,1,3,0,2
3,1,0,0,1
4,1,0,0,1
...,...,...,...,...
1333,1,3,0,1
1334,0,0,0,0
1335,0,0,0,2
1336,0,0,0,3


In [35]:
y.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [36]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [37]:
from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(x_train, y_train)

y_pred = mlr.predict(x_test)

In [38]:
import matplotlib.pyplot as plt

plt.scatter(x_train[:, 0], y_train, color='red')
plt.show

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [39]:
#Menggunakan One Hot Encoder#
from sklearn.metrics import r2_score
r2_mlr = r2_score(y_test, y_pred)
print(r2_mlr)

0.7866124253214011


#### (Hints) Interpretasi Nilai VIF

- 1 - variabel indenpenden tidak memiliki korelasi dengan variabel independen yang lain
- 1 < VIF < 5 - variabel independen sedikit memiliki korelasi dengan variabel independen yang lain
- VIF > 5 - variabel independen memiliki korelasi yang kuat dengan variabel independen lainnya
- VIF > 10 - variabel independen miliki korelasi yang sangat kuat dengan variabel independen dan perlu diperhatikan lebih lanjut

#### (Hints) Implementasi Perhitungan VIF

VIF dapat dihitung secara langsung dengan menggunakan library dari `statsmodels`

#### (Hints) Scatterplot Korelasi Antar Variabel

![var_cor](assets/var_corr.png)